# Resumable Index Creation in SQL Server 2019
This example will show you how to use resumable index builds in SQL Server 2019. This example includes all the steps you will need. You will need SQL Server 2019 Enterprise, Evaluation, or Developer Edition to complete this example

## Step 1: Setup the database
Use these T-SQL statements to create the database for the example

In [3]:
USE master
GO
DROP DATABASE IF EXISTS gotexasrangers
GO
CREATE DATABASE gotexasrangers
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.272

## Step 2: Create the table and insert data
Use these T-SQL statements to create a table and populate it with data<br>
NOTE: This could take a long time to populate a big enough table.

In [4]:
USE gotexasrangers
GO
DROP TABLE IF EXISTS letsgorangers
GO
CREATE TABLE letsgorangers (col1 int, col2 char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
GO
INSERT INTO letsgorangers values (1, 'I would love to win the World Series')
GO 750000
COMMIT TRAN
GO
SET NOCOUNT OFF
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Beginning execution loop

Batch execution completed 750000 times...

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:09:27.453

## Step 3: Create an online resumable index
Use the new feature for SQL Server 2019 to allow a CREATE INDEX to be online and resumable. Notice the use of the MAX_DURATION option of 1 minute. This means that the index will be PAUSED after 1 minute if has not completed. It will look like a failure but the index can be resumed at a later time.

In [5]:
CREATE CLUSTERED INDEX rangeridx ON letsgorangers (col1) WITH (ONLINE = ON, RESUMABLE = ON, MAX_DURATION = 1)
GO

: Msg 3643, Level 16, State 1, Line 1
The operation elapsed time exceeded the maximum time specified for this operation. The execution has been stopped.

The statement has been terminated.

: Msg 596, Level 21, State 1, Line 0
Cannot continue the execution because the session is in the kill state.

: Msg 0, Level 20, State 0, Line 0
A severe error occurred on the current command.  The results, if any, should be discarded.

Total execution time: 00:01:08.321

## Step 4: Check the progress of the paused index build using a DMV
Check the values. The index should be around 20-40% complete but PAUSED. Notice you have to change back to the right database context becasue the connection was terminated from the previous paused index.

In [6]:
USE gotexasrangers
GO
SELECT * FROM sys.index_resumable_operations
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.203

object_id,index_id,name,sql_text,last_max_dop_used,partition_number,state,state_desc,start_time,last_pause_time,total_execution_time,percent_complete,page_count
581577110,1,rangeridx,"CREATE CLUSTERED INDEX rangeridx ON letsgorangers (col1) WITH (ONLINE = ON, RESUMABLE = ON, MAX_DURATION = 1)",0,NULL,1,PAUSED,2019-07-13 20:26:54.877,2019-07-13 20:27:41.303,1,26.6666666666667,0


## Step 5: Resume the index build using ALTER INDEX
Let this finish to complete building the index

In [7]:
ALTER INDEX rangeridx on letsgorangers RESUME
GO

Commands completed successfully.

Total execution time: 00:01:30.584

## Step 6: Drop the index and set the default database options
Drop the existing index so we can try it another way. Set database default options for indexes to be automatically ONLINE and RESUMABLE should the index qualify for online and resumable.

In [8]:
USE gotexasrangers
GO
ALTER DATABASE SCOPED CONFIGURATION SET ELEVATE_RESUMABLE = WHEN_SUPPORTED
GO
ALTER DATABASE SCOPED CONFIGURATION SET ELEVATE_ONLINE = WHEN_SUPPORTED
GO
DROP INDEX IF EXISTS letsgorangers.rangeridx
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:47.358

## Step 7: Create the index again with no special options
Because you used the default database scoped configuration, the index will be created as ONLINE and RESUMABLE. **Hit the stop button for this cell after about 30 seconds**. This will PAUSE the index build.

In [9]:
CREATE CLUSTERED INDEX rangeridx ON letsgorangers (col1)
GO

The statement has been terminated.

Query was canceled by user

Total execution time: 00:00:15.207

## Step 8: Check the progress of the index build

In [10]:
USE gotexasrangers
GO
SELECT * FROM sys.index_resumable_operations
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.054

object_id,index_id,name,sql_text,last_max_dop_used,partition_number,state,state_desc,start_time,last_pause_time,total_execution_time,percent_complete,page_count
581577110,1,rangeridx,CREATE CLUSTERED INDEX rangeridx ON letsgorangers (col1),0,NULL,1,PAUSED,2019-07-13 20:30:54.797,2019-07-13 20:30:59.250,0,13.3333333333333,0


## Step 9: Resume the index build

In [11]:
ALTER INDEX rangeridx on letsgorangers RESUME
GO

Commands completed successfully.

Total execution time: 00:01:59.035